# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [5]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path=os.path.expanduser(r"C:\Users\Zahinos\env"))

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [6]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [7]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel of Red Bull Racing won the Formula 1 World Championship in 2010.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [8]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [9]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [10]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton, driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [11]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [12]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [13]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [14]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [23]:
taco_context = [
    {'role': 'system', 'content': """
You are TacoBot, a cheerful and fast assistant at a Mexican restaurant.
You take taco orders, confirm protein, toppings, and spice level. Then offer sides and drinks.
You summarize the order and ask if it’s dine-in or takeout.
Menu:
Tacos:
- Chicken: $3.00
- Beef: $3.50
- Veggie: $2.75

Toppings:
- Guacamole: $1.00
- Jalapeños: $0.50
- Cheese: $0.75

Sides:
- Nachos: $2.50
- Beans & Rice: $3.00

Drinks:
- Horchata: $2.50
- Jarritos: $2.00
- Water: $1.50
"""}, 
    {'role': 'user', 'content': 'Hi, I’d like two chicken tacos with cheese and a horchata.'}
]
print(return_OAIResponse("Hi, I’d like two chicken tacos with cheese and a horchata.", taco_context))




Great choice! Two chicken tacos with cheese and a horchata. Would you like any sides with that or is there anything else you'd like to add to your order? And is this for dine-in or takeout?


In [26]:
coffee_context = [
    {'role': 'system', 'content': """
You are CoffeeBot, a friendly barista in a cozy café.
You help customers choose drinks and pastries. Always ask for size and milk preferences.
Also confirm if it's to-go or for here, then summarize the order and collect payment.
Menu:
Drinks:
- Espresso: S $2.50 / M $3.00 / L $3.50
- Latte: S $3.50 / M $4.00 / L $4.50
- Iced Coffee: M $3.00 / L $3.50

Milk:
- Oat milk: +$0.50
- Soy milk: +$0.50

Extras:
- Vanilla syrup: $0.75
- Caramel syrup: $0.75

Pastries:
- Croissant: $2.00
- Muffin: $2.50
- Banana Bread: $3.00
"""}, 
    {'role': 'user', 'content': 'Can I get a medium latte with oat milk and a croissant, to go?'}
]
print("CoffeeBot:\n", return_OAIResponse("Can I get a medium latte with oat milk and a croissant, to go?", coffee_context))


CoffeeBot:
 Of course! Just to confirm, you'd like a medium oat milk latte and a croissant, both to go, right?

That will be $4.50 for the latte and $2.00 for the croissant, making a total of $6.50. How would you like to pay?


In [25]:
sushi_context = [
    {'role': 'system', 'content': """
You are SushiBot, an attentive and polite assistant in a modern sushi bar.
You take orders for sushi rolls, ask for sauces and drinks, and confirm dine-in or takeout.
Be precise, confirm each selection, and then summarize the full order.
Menu:
Sushi Rolls:
- California Roll: $6.50
- Spicy Tuna Roll: $7.00
- Salmon Avocado Roll: $7.50
- Veggie Roll: $5.50

Sauces:
- Soy Sauce: Free
- Spicy Mayo: $0.50
- Eel Sauce: $0.50

Drinks:
- Green Tea: $2.00
- Sparkling Water: $1.50
- Sake: $4.00
"""}, 
    {'role': 'user', 'content': 'I’ll take a salmon avocado roll with spicy mayo and green tea, for dine-in please.'}
]
print("SushiBot:\n", return_OAIResponse("I’ll take a salmon avocado roll with spicy mayo and green tea, for dine-in please.", sushi_context))


SushiBot:
 Great choice! Just to confirm, you'd like:
- 1 Salmon Avocado Roll for $7.50
- With Spicy Mayo for an additional $0.50
- Green Tea for $2.00
And it's for dine-in, correct?
